# LOGISTIC REGRESSION WITH MNIST

In [9]:
import numpy as np
# import tensorflow as tf
import tensorflow.compat.v1 as tf
import matplotlib.pyplot as plt
# tf.disable_eager_execution()
# tf.enable_eager_execution()
print ("PACKAGES LOADED")

PACKAGES LOADED


# DOWNLOAD AND EXTRACT MNIST DATASET

In [10]:
def OnehotEncoding(target):
    from sklearn.preprocessing import OneHotEncoder
    target_re = target.reshape(-1,1)
    enc = OneHotEncoder()
    enc.fit(target_re)
    return enc.transform(target_re).toarray()

In [11]:
def SuffleWithNumpy(data_x, data_y):
    idx = np.random.permutation(len(data_x))
    x,y = data_x[idx], data_y[idx]
    return x,y

## download with keras dataset

In [12]:
print ("Download and Extract MNIST dataset")
# mnist = input_data.read_data_sets('data/', one_hot=True)
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

print
print (" tpye of 'mnist' is %s" % (type(mnist)))
print (" number of train data is %d" % (len(x_train)))
print (" number of test data is %d" % (len(x_test)))
num_train_data = len(x_train)
trainimg   = x_train
trainimg = trainimg.reshape(len(trainimg),784)
trainlabel = OnehotEncoding(y_train)
testimg    = x_test
testimg = testimg.reshape(len(testimg),784)
testlabel  = OnehotEncoding(y_test)
print ("MNIST loaded")
tf.disable_eager_execution()

Download and Extract MNIST dataset
 tpye of 'mnist' is <class 'tensorflow.python.util.module_wrapper.TFModuleWrapper'>
 number of train data is 60000
 number of test data is 10000
MNIST loaded


## Download with tfds

In [3]:
import tensorflow_datasets as tfds
print ("Batch Learning? ")
tf.enable_eager_execution()
dataset, metadata = tfds.load('mnist', as_supervised=True, with_info=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

num_train_examples = metadata.splits['train'].num_examples
num_test_examples = metadata.splits['test'].num_examples
print("Number of training examples: {}".format(num_train_examples))
print("Number of test examples:     {}".format(num_test_examples))

def normalize(images, labels):
    images = tf.cast(images, tf.float32)
    images /= 255
    return images, labels

# The map function applies the normalize function to each element in the train
# and test datasets
train_dataset =  train_dataset.map(normalize)
test_dataset  =  test_dataset.map(normalize)

# The first time you use the dataset, the images will be loaded from disk
# Caching will keep them in memory, making training faster
train_dataset =  train_dataset.cache()
test_dataset  =  test_dataset.cache()

train_dataset=train_dataset.shuffle(num_train_examples,reshuffle_each_iteration=True)

train_to_np=tf.compat.v1.data.make_one_shot_iterator(train_dataset.batch(num_train_examples)).get_next()
x_train=train_to_np[0].numpy()
y_train=train_to_np[1].numpy()
test_to_np=tf.compat.v1.data.make_one_shot_iterator(test_dataset.batch(num_test_examples)).get_next()
x_test = test_to_np[0].numpy()
y_test = test_to_np[1].numpy()

num_train_data = len(x_train)
trainimg   = x_train
trainlabel = OnehotEncoding(y_train)
trainimg = trainimg.reshape(len(trainimg),784)
testimg    = x_test
testimg = testimg.reshape(len(testimg),784)
testlabel  = OnehotEncoding(y_test)

tf.disable_eager_execution()
trainimg.shape, trainlabel.shape

Batch Learning? 
Number of training examples: 60000
Number of test examples:     10000


((60000, 28, 28, 1), (60000, 10))

## CREATE TENSOR GRAPH FOR LOGISTIC REGRESSION

In [13]:
x = tf.placeholder(tf.float32, shape = (None, 784))
y = tf.placeholder(tf.float32, shape = (None, 10))  # None is for infinite 
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
# LOGISTIC REGRESSION MODEL
actv = tf.nn.softmax(tf.matmul(x, W) + b) 
# COST FUNCTION
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.math.log(actv), axis=1)) 
# OPTIMIZER
learning_rate = 0.01
optm = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

## PREDICTION AND ACCURACY

In [14]:
# PREDICTION
pred = tf.equal(tf.argmax(actv, 1), tf.argmax(y, 1))    
# ACCURACY
accr = tf.reduce_mean(tf.cast(pred, "float"))
# INITIALIZER
initializer = tf.global_variables_initializer()

# TRAIN MODEL

In [15]:
training_epochs = 50
batch_size      = 100
display_step    = 5
# SESSION
sess = tf.Session()
sess.run(initializer)
# MINI-BATCH LEARNING
for epoch in range(training_epochs):
    avg_cost = 0.
    num_batch = int(num_train_data/batch_size)
    for i in range(num_batch):         
        batch_xs=trainimg[i*batch_size:(i+1)*batch_size]
        batch_ys=trainlabel[i*batch_size:(i+1)*batch_size]

        sess.run(optm, feed_dict={x: batch_xs, y: batch_ys})
        feeds = {x: batch_xs, y: batch_ys}
        avg_cost += sess.run(cost, feed_dict=feeds)/num_batch        
    # DISPLAY
    if epoch % display_step == 0:
        feeds_train = {x: batch_xs, y: batch_ys}
        feeds_test = {x: testimg, y: testlabel}
        train_acc = sess.run(accr, feed_dict=feeds_train)
        test_acc = sess.run(accr, feed_dict=feeds_test)
        print ("Epoch: %03d/%03d cost: %.9f train_acc: %.3f test_acc: %.3f" 
               % (epoch, training_epochs, avg_cost, train_acc, test_acc))
    #shuffle in each epoch
    trainimg,trainlabel = SuffleWithNumpy(trainimg,trainlabel)
    
print ("DONE")

Epoch: 000/050 cost: 1.137559623 train_acc: 0.830 test_acc: 0.856
Epoch: 005/050 cost: 0.429877024 train_acc: 0.870 test_acc: 0.896
Epoch: 010/050 cost: 0.375152144 train_acc: 0.950 test_acc: 0.905
Epoch: 015/050 cost: 0.350435106 train_acc: 0.920 test_acc: 0.909
Epoch: 020/050 cost: 0.335372693 train_acc: 0.890 test_acc: 0.913
Epoch: 025/050 cost: 0.324955257 train_acc: 0.910 test_acc: 0.915
Epoch: 030/050 cost: 0.317174830 train_acc: 0.930 test_acc: 0.916
Epoch: 035/050 cost: 0.311087956 train_acc: 0.910 test_acc: 0.918
Epoch: 040/050 cost: 0.306114847 train_acc: 0.870 test_acc: 0.918
Epoch: 045/050 cost: 0.301994932 train_acc: 0.930 test_acc: 0.919
DONE
